In [19]:
a=torch.zeros([8,2], dtype=torch.float32)

In [5]:
coordinates

[{'x_min': 522, 'y_min': 138, 'x_max': 591, 'y_max': 393},
 {'x_min': 943, 'y_min': 466, 'x_max': 1013, 'y_max': 720}]

In [7]:
import xml.etree.ElementTree as ET
def bboxParser(fileName):
    root = ET.parse(fileName + '.xml')
    coordinates = []
    for obj in root.iter('HRSC_Object'):
        coordinate = {}
        coordinate['x_min'] = int(obj.find('box_xmin').text)
        coordinate['y_min'] = int(obj.find('box_ymin').text)
        coordinate['x_max'] = int(obj.find('box_xmax').text)
        coordinate['y_max'] = int(obj.find('box_ymax').text)
        coordinates.append(coordinate)
        
    return int(root.find('Img_SizeWidth').text), int(root.find('Img_SizeHeight').text), coordinates

In [17]:
import xml.etree.ElementTree as ET
def objectParser(fileName):
    try:
        root = ET.parse(fileName)
        object_num = 0
        for obj in root.iter('HRSC_Object'):
            object_num = object_num + 1
        return object_num
    except:
        pass

In [19]:
object_num_list = []
for i in range(1,1681):
    name = './dataset/labels/1' + str(i).zfill(8) + '.xml'
    object_num = objectParser(name)
    if object_num != None:
        object_num_list.append(object_num)
        
print(object_num_list)

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 0, 1, 0, 1, 1, 1, 0, 1, 4, 3, 0, 1, 1, 5, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 0, 2, 3, 1, 3, 3, 2, 1, 1, 2, 1, 1, 5, 0, 4, 1, 0, 1, 1, 4, 3, 2, 3, 1, 2, 2, 4, 2, 2, 3, 2, 5, 2, 1, 1, 2, 2, 3, 1, 1, 1, 8, 3, 3, 0, 5, 4, 5, 4, 4, 3, 1, 6, 1, 4, 3, 2, 1, 1, 1, 2, 1, 4, 2, 3, 3, 2, 1, 1, 5, 5, 2, 1, 1, 1, 1, 1, 1, 6, 4, 1, 1, 1, 5, 3, 3, 1, 1, 1, 1, 1, 5, 1, 2, 1, 1, 5, 2, 1, 3, 1, 1, 3, 4, 5, 3, 1, 2, 1, 1, 1, 2, 1, 4, 1, 1, 1, 1, 2, 6, 2, 6, 4, 5, 4, 2, 1, 1, 3, 1, 3, 2, 1, 1, 1, 1, 1, 0, 2, 2, 5, 1, 1, 1, 3, 2, 1, 1, 2, 2, 1, 4, 4, 1, 2, 2, 1, 3, 2, 2, 1, 1, 1, 4, 2, 2, 2, 3, 1, 3, 3, 1, 5, 1, 3, 2, 5, 1, 2, 1, 2, 2, 4, 4, 3, 3, 1, 1, 1, 1, 4, 3, 1, 2, 1, 2, 3, 2, 2, 2, 2, 2, 1, 3, 7, 2, 1, 1, 1, 1, 3, 2, 3, 1, 1, 1, 3, 3, 5, 1, 3, 5, 6, 1, 2, 3, 2, 3, 1, 1, 2, 2, 3, 5, 6, 8, 3, 1, 1, 1, 3, 1, 2, 3, 4, 5, 3, 8, 9, 1, 2, 1, 2, 5, 3, 6, 2, 4, 3, 3, 6, 2, 3, 1, 1, 1, 2, 3, 1, 2, 1, 2, 2, 8, 4, 3, 1, 6, 3, 9, 5, 1, 3, 1, 2, 1, 1, 3, 7, 4, 6, 1, 1, 1, 

In [20]:
len(object_num_list)

626

In [5]:
import torch
def weight(shape):
    w = torch.empty(shape)
    initial = torch.nn.init.trunc_normal_(w)
    return initial

In [6]:
w = weight((3,5))

In [7]:
w

tensor([[-0.0562,  0.1950,  0.2111, -0.7356,  0.4723],
        [-0.5127,  0.7964,  0.8249, -1.7378,  0.3028],
        [ 0.2230, -0.6746, -1.2672, -1.6923, -1.3758]])

In [ ]:
import cv2 as cv
import numpy as np
max_h = 0
max_w = 0
for i in range(1,1681):
    name = './current_data/AllImages/1' + str(i).zfill(8) + '.bmp'
    img = cv.imread(name)
    if img is None:
        pass
    else:
        img_h, img_w, ch = img.shape
        print(name)
        if img_h > max_h:
            max_h = img_h
        if img_w > max_w:
            max_w = img_w
print('max_h: %d, max_w: %d' % (max_h, max_w))

In [6]:
import cv2 as cv
import numpy as np
new_h = 844
new_w = 1238
for i in range(1,1681):
    name = './current_data/AllImages/1' + str(i).zfill(8) + '.bmp'
    img = cv.imread(name)
    if img is None:
        pass
    else:
        img_h, img_w, ch = img.shape
        padding = np.full((new_h,new_w,3),(34,47,32), dtype=np.uint8)
        padding[0:img_h, 0:img_w] = img
        cv.imwrite('./pad/'+str(i)+'.bmp', padding)


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import DataLoader, Dataset

dataset = torchvision.datasets.ImageFolder('./dataset/',transform=transforms.Compose([transforms.ToTensor()]))


FileNotFoundError: Found no valid file for the classes labels. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [ ]:
from PIL import Image
from torchvision import transforms
imgs = []
tran = transforms.Compose([transforms.CenterCrop((224,224)),
                           transforms.ToTensor()])
img = Image.open('./dataset/data/1.bmp').convert('RGB')
img_tensor = tran(img)
for i in range(5):
    imgs.append(img_tensor)
print(imgs)

TypeError: 'torch.Size' object is not callable

In [61]:
a = torch.ones(128,1,8,8, dtype=torch.int)

In [ ]:
for i in a:
    print(i)

In [51]:
b = torch.tensor([[1,2,3],[4,5,6]])

In [54]:
b.shape

torch.Size([2, 3])

In [59]:
import torch.nn.functional as F
b = F.pad(a, (2,2,2,2))

In [60]:
b.shape

torch.Size([128, 1, 32, 32])

In [62]:
c =torch.cat(a, 1)

TypeError: cat() received an invalid combination of arguments - got (Tensor, int), but expected one of:
 * (tuple of Tensors tensors, name dim, *, Tensor out)
 * (tuple of Tensors tensors, int dim, *, Tensor out)


In [64]:
c = a.view(a.shape[0], -1)

In [66]:
d = torch.ones(3)
e = torch.ones(3)

In [67]:
f = d+e

In [68]:
f.shape

torch.Size([3])

In [69]:
for i in f:
    print(i)

tensor(2.)
tensor(2.)
tensor(2.)


In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal

a = torch.arange(10, dtype=torch.float)
b = Normal(0.5, 0.01).sample()

In [34]:
a

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

In [103]:
for i in range(10):
    a = Normal(0.5, 0.22).rsample()
    print(Normal(0.5, 0.22).log_prob(a))

tensor(0.3965)
tensor(0.5616)
tensor(0.5653)
tensor(0.5952)
tensor(-0.4507)
tensor(0.5311)
tensor(0.1808)
tensor(0.5786)
tensor(0.1674)
tensor(0.4990)
